In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import languagetool as lt
from spacy import load
nlp = load('en_core_web_md')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2023-01-24 14:54:44.366994: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 14:54:45.471225: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-24 14:54:45.471280: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-24 14:54:45.471285: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [3]:
from joblib import Parallel, delayed

In [4]:
for df in pd.read_json('/tmp/exports.json', lines=True, chunksize=1000):
    errs = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(**row) for _, row in df.iterrows())
    break

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
2023-01-24 14:56:09.708893: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-24 14:56:09.709271: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-24 14:56:09.709284: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-24 14:56:09.743880: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror

In [6]:
errs = [e for e in errs if any([c['rule']['id'].startswith('SUBJECT_VERB_AGREEMENT_') for c in e['matches']])]

In [8]:
errs[0]

{'question_id': 354,
 'matches': [{'message': 'The verb form ‘is’ does not seem to match the subject ‘exports’.',
   'shortMessage': '',
   'replacements': [{'value': 'are'},
    {'value': 'was'},
    {'value': 'were',
     'shortDescription': "second-person singular simple past of 'be', e.g. 'you were'"},
    {'value': 'will be'}],
   'offset': 49,
   'length': 2,
   'context': {'text': '...um merchandise exports across all years is 81.125026324.',
    'offset': 43,
    'length': 2},
   'sentence': 'The maximum merchandise exports across all years is 81.125026324.',
   'type': {'typeName': 'Other'},
   'rule': {'id': 'SUBJECT_VERB_AGREEMENT_PLURAL',
    'subId': '2',
    'sourceFile': 'grammar.xml',
    'description': 'Subject-verb agreement plural',
    'issueType': 'grammar',
    'category': {'id': 'GRAMMAR', 'name': 'Grammar'}},
   'ignoreForIncompleteSentence': True,
   'contextForSureMatch': 10}]}

In [10]:
doc = nlp(errs[0]['matches'][0]['sentence'])
doc

The maximum merchandise exports across all years is 81.125026324.

In [15]:
nouns = [t for t in doc if t.tag_ in ('NN', 'NNS')]

In [16]:
nouns

[merchandise, exports, years]

In [18]:
n1, n2, n3 = nouns

In [20]:
list(n1.ancestors)

[exports, is]

In [22]:
list(n2.ancestors)

[is]

In [23]:
list(n3.ancestors)

[across, exports, is]

In [24]:
from spacy import explain

In [25]:
explain('AUX')

'auxiliary'

In [26]:
aux = [k for k in n2.ancestors][0]

In [27]:
aux

is

In [29]:
aux.tag_

'VBZ'

In [30]:
aux.pos_

'AUX'

In [31]:
explain('VBZ')

'verb, 3rd person singular present'

In [32]:
for noun in nouns:
    for anc in noun.ancestors:
        if anc.pos_ == 'AUX':
            break

In [33]:
noun

years

In [34]:
anc

is

In [35]:
aux.dep_

'ROOT'

In [37]:
list(aux.children)

[exports, 81.125026324, .]

In [38]:
exp = _[0]

In [39]:
exp

exports

In [40]:
exp.dep_

'nsubj'

In [42]:
list(aux.children)[1]

81.125026324

In [43]:
_.dep_

'attr'

In [151]:
def find_root_nsubj(doc):
    root = None
    for token in doc:
        if token.dep_ == 'ROOT':
            root = token
            break
    nsubj = None
    for noun in root.children:
        if noun.dep_ == 'nsubj':
            nsubj = noun
            break
    return root, nsubj

is_noun_singular = lambda x: x.tag_ in ('NN', 'NNP')
is_noun_plural   = lambda x: x.tag_ in ('NNS', 'NNPS')


def has_agreement(verb, subject):
    if (subject is None) or (subject.tag_ == 'CD'):
        return True
    if verb.tag_ == 'VBP':
        return is_noun_plural(subject)
    if verb.tag_ == 'VBZ':
        return is_noun_singular(subject)
    if verb.tag_ == 'VBD':
        is_singular = verb.morph.get('Number') == ['Sing']
        if is_singular:
            return is_noun_singular(subject)
        return is_noun_plural(subject)
    if verb.tag_ == 'VB':
        # Find the auxiliary child and that becomes the new verb
        for child in verb.children:
            if child.dep_ == 'aux':
                break
        return has_agreement(child, subject)
    raise ValueError(f'Tag for verb {verb} not valid.')
    
    
def has_verb_subject_agreement(doc):
    return has_agreement(*find_root_nsubj(doc))

In [55]:
c = 0
for err in errs:
    sentence = err['matches'][0]['sentence']
    doc = nlp(sentence)
    verb, subject = find_root_nsubj(doc)
    if not has_agreement(verb, subject):
        c += 1

In [58]:
len(errs), c

(166, 166)

In [59]:
df.shape

(1000, 7)

In [60]:
df['has_agreement'] = True

In [62]:
df.set_index('question_id', verify_integrity=True, inplace=True)
df.loc[[c['question_id'] for c in errs], 'has_agreement'] = False

In [156]:
c = []
for ix, x in df['caption'].items():
    doc = nlp(x)
    if not has_verb_subject_agreement(doc):
        c.append(ix)

In [157]:
len(c)

460

In [158]:
noag = df[~df['has_agreement']]

In [159]:
noag.shape

(166, 7)

In [161]:
amb = [k for k in c if k not in noag.index]

In [163]:
amb = df.loc[amb]

In [167]:
amb['caption'].sample(5).tolist()

['The merchandise exports in 1990 is less than that in 1995.',
 '5000000 is the difference between the arms exports in 1960 and that in 2000.',
 'The arms exports was the minimum in the 1989 year.',
 '17.5269767991 is the total merchandise exports in the graph.',
 '2.6635391436 is the difference between the earnings from merchandise imports in 1990 and the earnings from merchandise exports in 1991.']

In [168]:
doc = nlp('5000000 is the difference between the arms exports in 1960 and that in 2000.')
verb, subject = find_root_nsubj(doc)

In [169]:
verb

is

In [170]:
subject

5000000

In [171]:
subject.tag_

'CD'

In [172]:
was

was

In [173]:
was.set_morph?

Docstring: Token.set_morph(self, features)
Type:      builtin_function_or_method

In [174]:
were.morph.to_dict()

{'Mood': 'Ind', 'Tense': 'Past', 'VerbForm': 'Fin'}

In [175]:
was.morph.to_dict()

{'Mood': 'Ind',
 'Number': 'Sing',
 'Person': '3',
 'Tense': 'Past',
 'VerbForm': 'Fin'}

In [183]:
t = nlp.vocab.lookups.get_table('lexeme_norm')

In [192]:
doc.vocab

In [193]:
nlp.vocab

In [190]:
i.text

'nuthin'

In [203]:
lemma = nlp.get_pipe('lemmatizer')

In [210]:
lemma.is_base_form(were)

False

In [211]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [365]:
docs = nlp.pipe(df['caption'].values)
newdocs = []
for i, doc in enumerate(docs):
    newdocs.append(lt.fix_verb_subject_agreement(doc))

    
newnewdocs = []
for i, doc in enumerate(newdocs):
    newnewdocs.append(lt.fix_verb_subject_agreement(nlp(doc.text)))

In [ ]:
res = Parallel(n_jobs=-1, verbose=2)(delayed(lt.process)(None, k.text) for k in newnewdocs)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


In [342]:
msgs = []
for i, k in enumerate(res):
    for match in k['matches']:
        msgs.append(match['message'])
ctr = Counter(msgs)
ctr

Counter({'The verb form ‘does’ does not seem to match the subject ‘exports’.': 16,
         'Possible spelling mistake found.': 1,
         'A determiner may be missing.': 1,
         'You should probably use “export”.': 3})

In [346]:
doc = nlp('The merchandise exports does not monotonically increase over the years.')
verb, subject = lt.find_root_nsubj(doc)

In [347]:
verb, subject

(increase, exports)

In [350]:
agrees, verb = lt.has_agreement(verb, subject)

In [351]:
agrees

False

In [352]:
verb

does

In [353]:
subject.tag_

'NNS'

In [355]:
lt.fix_verb_subject_agreement(doc)

The merchandise exports do not monotonically increase over the years.